In [1]:
import torch
import pickle
from EncoderDecoderAttentionModel import Seq2Seq

state_dict = torch.load('seq2seq_state.bin')

with open('./train_data/vocabs', 'r') as f:
    lines = f.readlines()
    tokens = ['PAD', 'UNK', 'BOS', 'EOS'] + [line.strip() for line in lines if line.strip()]
    vocab = { tk:i for i, tk in enumerate(tokens)}



In [3]:
model = Seq2Seq(
    enc_emb_size=len(vocab),
    dec_emb_size=len(vocab),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.load_state_dict(state_dict)

e:\miniconda3\envs\py3127\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


<All keys matched successfully>

In [6]:
# 创建解码器反向字典
dvoc_inv = {v:k for k,v in vocab.items()}

In [5]:
evoc, dvoc = vocab, vocab

In [11]:
# 用户输入
enc_input = "几 树 梅 花 数 竿 竹"
# enc_input = "What I'm about to say is strictly between you and me"
# enc_input = "I used to go swimming in the sea when I was a child"
enc_idx = torch.tensor([[evoc[tk] for tk in enc_input.split()]])


# 推理
# 最大解码长度
max_dec_len = 50

model.eval()
with torch.no_grad():
    # 编码器
    encoder_state, enc_outputs = model.encoder(enc_idx)
    # hidden_state, enc_outputs = model.encoder(enc_idx)  # attention

    # 解码器输入 shape [1,1]
    dec_input = torch.tensor([[dvoc['BOS']]])

    # 循环decoder
    dec_tokens = []
    while True:
        if len(dec_tokens) >= max_dec_len:
            break
        # 解码器 
        # logits: [1,1,dec_voc_size]
        logits,hidden_state = model.decoder(dec_input, encoder_state, enc_outputs)
        
        # 下个token index
        next_token = torch.argmax(logits, dim=-1)

        if dvoc_inv[next_token.squeeze().item()] == 'EOS':
            break
        # 收集每次token_index 【解码集合】
        dec_tokens.append(next_token.squeeze().item())
        # decoder的下一个输入 = token_index
        dec_input = next_token
        hidden_state = hidden_state.view(1, -1)

# 输出解码结果
print(''.join([dvoc_inv[tk] for tk in dec_tokens]))

一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪水一溪
